In [1]:
# !wget "https://raw.githubusercontent.com/00ber/ml-projects/main/data/weight_diff.txt"

In [2]:
## Neural Network model + Training code 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

class PUFWeightsPredictor(nn.Module):
    
    def __init__(self, input_dims=64, output_dims=1, weights_init=None):
        super(PUFWeightsPredictor, self).__init__()
        self.input_dims = input_dims
        self.output_dims = output_dims

        self.fc = nn.Linear(input_dims + 1, 1, bias=False)
        self.training = True
        if isinstance(weights_init, tuple):
          nn.init.uniform_(self.fc.weight, a=weights_init[0], b=weights_init[1])
        elif weights_init is not None:
          nn.init.normal_(self.fc.weight, mean=0.0, std=weights_init)

    def forward(self, phi):
      return self.fc(phi)

    def get_weights(self):
      state = self.state_dict().copy()
      wt = state["fc.weight"]
      wt = wt.reshape(self.input_dims + 1, 1).cpu().numpy()
      return wt

class Trainer:
  def __init__(self, model, num_epochs, lr, batch_size, train_ds, val_ds):
    self.model = model
    self.num_epochs = num_epochs
    self.lr = lr 
    self.batch_size = batch_size
    self.train_ds = train_ds
    self.val_ds = val_ds
    self.train_dl = DataLoader(
        self.train_ds, 
        batch_size=batch_size, 
        shuffle=True
    )
    self.val_dl = DataLoader(
        self.val_ds, 
        batch_size=batch_size, 
        shuffle=True
    )
   

  def train(self):
    criterion = nn.BCEWithLogitsLoss()
    # optimizer = Adam(self.model.parameters(), lr=self.lr)
    optimizer = SGD(self.model.parameters(), lr=self.lr)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    self.model = self.model.to(device)
    print("Begin training...") 

    for epoch in range(1, self.num_epochs + 1): 
        running_train_loss = 0.0 
        running_val_loss = 0.0 

        # Training Loop 
        self.model.train()
        for train_batch in self.train_dl:
            inputs, outputs = train_batch  
            inputs = inputs.to(device)
            outputs = outputs.to(device)
            optimizer.zero_grad()   
            predicted_outputs = self.model(inputs)  
            predicted_outputs = predicted_outputs.to(device)
            train_loss = criterion(predicted_outputs, outputs) 
            train_loss.backward()   
            optimizer.step()   

            running_train_loss += train_loss.item() 
            
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(self.train_dl) 

        # Validation Loop 
        with torch.no_grad(): 
            self.model.eval() 
           
            for val_batch in self.val_dl:
               inputs, outputs = val_batch 
               inputs = inputs.to(device)
               outputs = outputs.to(device)
               predicted_outputs = self.model(inputs) 
               predicted_outputs = predicted_outputs.to(device)
               val_loss = criterion(predicted_outputs, outputs) 
               running_val_loss += val_loss.item()   
               
        # Calculate validation loss value 
        val_loss_value = running_val_loss/len(self.val_dl) 
 
        # Print the statistics of the epoch 
        print(f"Epoch {epoch}/{self.num_epochs} Avg. training loss: {train_loss_value:.4f} Avg. val loss: {val_loss_value:.4f}")

In [3]:
import numpy as np
import time
from sklearn.model_selection import train_test_split

def puf_query(c, w):
    n = c.shape[1]
    phi = np.ones(n+1)
    phi[n] = 1
    for i in range(n-1, -1, -1):
        phi[i] = (2*c[0,i]-1)*phi[i+1]

    r = (np.dot(phi, w) > 0)
    return r
    
# Problem Setup
target = 0.99  # The desired prediction rate
n = 64  # number of stages in the PUF

# Initialize the PUF
np.random.seed(int(time.time()))
data = np.loadtxt('weight_diff.txt')
w = np.zeros((n+1, 1))
for i in range(1, n+2):
    randi_offset = np.random.randint(1, 45481)
    w[i-1] = data[randi_offset-1]

# Syntax to query the PUF:
c = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
r = puf_query(c, w)
# you may remove these two lines

# You can use the puf_query function to generate your training dataset
# ADD YOUR DATASET GENERATION CODE HERE
training_size = 10000
X = np.random.randint(0, 2, size=(training_size, n))
y = np.zeros((training_size, 1))

for i in range(training_size):
  y[i] = puf_query(X[i].reshape(1, -1), w)

def calc_phi(select_bits):
    phi_vals = []
    for i in range(len(select_bits)):
      target_slice = select_bits[i:]
      zeros = [z for z in target_slice if z == 0]
      phi = 1 if len(zeros) % 2 == 0 else -1
      phi_vals.append(phi)
    return np.array(phi_vals + [1])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_phi = np.apply_along_axis(calc_phi, 1, X_train)
X_test_phi = np.apply_along_axis(calc_phi, 1, X_test)

In [4]:

def train_and_eval(params):
  w0 = np.zeros((n+1, 1))  # The estimated value of w.
  # Try to estimate the value of w here. This section will be timed. You are
  # allowed to use the puf_query function here too, but it will count towards
  # the training time.
  results = []
  lr = params["lr"]
  num_epochs = params["num_epochs"] 
  weight_init = params["weight_init"]
  training_size = params["training_size"] 

  train_dataset = TensorDataset(torch.from_numpy(X_train_phi[:training_size]).float(), torch.from_numpy(y_train[:training_size]).float())
  val_dataset = TensorDataset(torch.from_numpy(X_test_phi).float(), torch.from_numpy(y_test).float())

  t0 = time.process_time()

  # ADD YOUR TRAINING CODE HERE
  

  model = PUFWeightsPredictor(n, 1, weight_init)
  trainer = Trainer(
      model=model, 
      num_epochs=num_epochs, 
      lr=lr,
      batch_size=training_size, 
      train_ds=train_dataset, 
      val_ds=val_dataset
  )
  trainer.train()

  t1 = time.process_time()
  training_time = t1 - t0  # time taken to get w0
  print("Training time:", training_time)
  print("Training size:", training_size)


  w0 = model.get_weights()

  # Evaluate your result
  n_test = 10000
  correct = 0
  for i in range(1, n_test+1):
      c_test = np.random.randint(0, 2, size=(1, n))  # a random challenge vector
      r = puf_query(c_test, w)
      r0 = puf_query(c_test, w0)
      correct += (r==r0)

  success_rate = correct/n_test
  print("Success rate:", success_rate)


  # If the success rate is less than 99%, a penalty time will be added
  # One second is add for each 0.01% below 99%.
  effective_training_time = training_time
  if success_rate < 0.99:
      effective_training_time = training_time + 10000*(0.99-success_rate)
  print("Effective training time:", effective_training_time)
  results.append({
        "lr": lr,
        "num_epochs": num_epochs,
        "weight_init": weight_init,
        "training_size": training_size,
        "training_time": training_time,
        "effective_training_time": effective_training_time[0] if isinstance(effective_training_time, np.ndarray) else effective_training_time,
        "success_rate": success_rate[0]
  })
  return results 

In [5]:
# params_grid = [
#     {
#       "lr": 300,
#       "num_epochs": 10,
#       "weight_init": (-30, 30),
#       "training_size": 10000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-20, 20),
#       "training_size": 10000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-10, 10),
#       "training_size": 10000,
#     },
#     {
#       "lr": 250,
#       "num_epochs": 10,
#       "weight_init": 1,
#       "training_size": 10000,
#     },
    
#     {
#       "lr": 300,
#       "num_epochs": 10,
#       "weight_init": (-30, 30),
#       "training_size": 5000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-20, 20),
#       "training_size": 5000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-10, 10),
#       "training_size": 5000,
#     },
#     {
#       "lr": 250,
#       "num_epochs": 10,
#       "weight_init": 1,
#       "training_size": 5000,
#     },

#     {
#       "lr": 300,
#       "num_epochs": 10,
#       "weight_init": (-30, 30),
#       "training_size": 15000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-20, 20),
#       "training_size": 15000,
#     },
#     {
#       "lr": 200,
#       "num_epochs": 10,
#       "weight_init": (-10, 10),
#       "training_size": 15000,
#     },
#     {
#       "lr": 250,
#       "num_epochs": 10,
#       "weight_init": 1,
#       "training_size": 15000,
#     },
# ]
# results = []
# for params in params_grid:
#   res = train_and_eval(params)
#   for r in res:
#     results.append(r)

# print(results)

In [6]:
# import pandas as pd
# results_df = pd.DataFrame.from_dict(results)
# pd.options.display.max_rows = 4000
# results_df.sort_values(by=["success_rate"], ascending=False).head(100)

In [7]:
params = {
    "lr": 200,
    "num_epochs": 10,
    "weight_init": (-10, 10),
    "training_size": 10000,
}
results = train_and_eval(params)

Begin training...
Epoch 1/10 Avg. training loss: 18.8087 Avg. val loss: 2.8482
Epoch 2/10 Avg. training loss: 2.8229 Avg. val loss: 0.1360
Epoch 3/10 Avg. training loss: 0.0908 Avg. val loss: 0.0461
Epoch 4/10 Avg. training loss: 0.0176 Avg. val loss: 0.0223
Epoch 5/10 Avg. training loss: 0.0090 Avg. val loss: 0.0187
Epoch 6/10 Avg. training loss: 0.0069 Avg. val loss: 0.0155
Epoch 7/10 Avg. training loss: 0.0063 Avg. val loss: 0.0148
Epoch 8/10 Avg. training loss: 0.0060 Avg. val loss: 0.0139
Epoch 9/10 Avg. training loss: 0.0059 Avg. val loss: 0.0136
Epoch 10/10 Avg. training loss: 0.0058 Avg. val loss: 0.0134
Training time: 0.234375
Training size: 10000
Success rate: [0.9961]
Effective training time: 0.234375
